In [1]:
import pandas as pd

In [2]:
# ---------------
# 偽裝成瀏覽器
# ---------------
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0.1) Gecko/2010010' \
    '1 Firefox/4.0.1',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':'en-us,en;q=0.5',
    'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.7'}
# ---------------
# 開啟有驗證碼的網頁
# ---------------
import requests
rs = requests.session()

def get_verify_code(rs):
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsMenu.aspx', stream = True, verify = False, headers = headers, timeout=None )
    # ---------------
    # 從網頁中抓取一些關鍵字
    # ---------------
    import re
    
    # get view state
    viewstate = re.search( 'VIEWSTATE"\s+value=.*=', res.text )
    viewstate = viewstate.group()[18:]
    
    # get eventvalidation
    eventvalidation = re.search( 'EVENTVALIDATION"\s+value=.*\w', res.text )
    eventvalidation = eventvalidation.group( )[ 24: ]
    
    return viewstate, eventvalidation

viewstate, eventvalidation = get_verify_code(rs)

print('----')
print('viewstate', viewstate)
print('----')
print('eventvalidation',eventvalidation)
print('----')

# ---------------
# 爬取資料
# ---------------

def get_data(rs, stock_id, viewstate, eventvalidation):
    import time
    payload = {
    '__EVENTTARGET':'',
    '__EVENTARGUMENT':'',
    '__LASTFOCUS':'',
    '__VIEWSTATE' : viewstate,                      #encode_viewstate[:-1],
    '__EVENTVALIDATION' : eventvalidation,          #encode_eventvalidation[:-1],
    'RadioButton_Normal' : 'RadioButton_Normal',
    'TextBox_Stkno' : stock_id,
    'CaptchaControl1 ' : 'Z67YB',
    'btnOK' : '%E6%9F%A5%E8%A9%A2',
    }

    rs.post( "http://bsr.twse.com.tw/bshtm/bsMenu.aspx", data=payload, headers=headers)#, verify = False, stream = True )
    time.sleep(1)
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsContent.aspx')#,verify = False ,stream = True, )
    return res

res = get_data(rs, '1101', viewstate, eventvalidation)
res.encoding='big5'
open('test.html', 'w', encoding='utf-8').write(res.text)


import pandas as pd
from io import StringIO

def parse_data(text):
    lines = text.split('\n')
    lines = [l for l in lines if len(l.split(',')) == 11]
    df = pd.read_csv(StringIO('\n'.join(lines)))

    first_df = df[df.columns[:5]]
    second_df = df[df.columns[6:]]
    second_df.columns = second_df.columns.str.replace('.1', '')
    final_df = first_df.append(second_df).set_index('序號').sort_index().dropna()
    return final_df


df = parse_data(res.text)

----
viewstate /wEPDwUJOTAxMjkxMjg3D2QWAgIDD2QWBAIBD2QWAmYPZBYEZg9kFgJmDxAPFgYeB0NoZWNrZWRnHglGb3JlQ29sb3IKAB4EXyFTQgIEZGRkZAIBD2QWAgIBDw8WBB8BCgAfAgIEZGQCAw9kFgQCAw8PFgIeBFRleHRlZGQCCQ8PFgIeB1Zpc2libGVoZGQYAgUeX19Db250cm9sc1JlcXVpcmVQb3N0QmFja0tleV9fFgMFElJhZGlvQnV0dG9uX05vcm1hbAUQUmFkaW9CdXR0b25fRXhjZAUQUmFkaW9CdXR0b25fRXhjZAUPQ2FwdGNoYUNvbnRyb2wxDwUkMzhkM2Y2ZjAtYzU1YS00MzJlLWJkMDQtMDkxMzg2NDY0M2QyZHzakAdWq2ToOKb4gKPFeV8=
----
eventvalidation /wEdAAYwSG/G9eu3z99JL1f59EVqfMmuxAJNAJcNkRRsVeJwwqKURZav/+YrVMqaWE2hvMxJwK4Ohf3nRgXHkFhQen1PRSciNdPYWmINCrip1wqw01PJCA0uL9aE2sjICZqv6GrFrSxO+rYTXiEO52vYWDHF
----


## 查詢券商進出資訊，使用total可以查詢個股，使用確認可以查看買賣超券商

In [33]:
buy = df['買進股數'].astype(int).groupby(df['券商']).sum()
sell= df['賣出股數'].astype(int).groupby(df['券商']).sum()
total = buy - sell
ls = []
for i in range(0,15):
    ls.append(i)
ls = pd.DataFrame(ls)
# 買方top 15
buyside = total.nlargest(15)
# 賣方 top 15
sellside = total.nsmallest(15)
buyside = pd.DataFrame(buyside,columns=["買超數"] )
buyside['買超券商'] = buyside.index
buyside.index = ls.index
sellside = pd.DataFrame(sellside,columns=["賣超數"])
sellside['賣超券商'] = sellside.index
sellside.index = ls.index
pd.concat([buyside,sellside],axis=1)

,買超數,買超券商,賣超數,賣超券商
0,559000,1650瑞銀,-2258000,9100群益
1,522000,9623富邦台北,-814000,1520瑞士信貸
2,450000,9A93永豐大稻,-772000,8960上海匯豐
3,437000,1530德意志,-631000,1440美林
4,409976,7000兆豐,-325000,8440摩根大通
5,206000,1020合 庫,-309000,8150台 新
6,205000,592P元富同大,-195600,1480美商高盛
7,194000,1470台灣摩根,-96960,9692富邦嘉義
8,150000,9815元大大灣,-95000,1045臺銀新竹
9,149000,1360港麥格理,-78906,9800元大


In [37]:
# 主力買賣超
judge = total.nlargest(15).sum() + total.nsmallest(15).sum()
if judge > 0:
    print("市場目前偏向多頭")
else:
    print("市場目前偏向空頭")
print(judge)

市場目前偏向空頭
-2028490


In [38]:
# 買賣家數差
diff = (total > 0).sum() - (total < 0).sum() 
print("買賣家數差為" , diff + message)

買賣家數差為 275
